In [22]:
import pandas as pd
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

In [23]:
egrul = pd.read_excel("Data/egrul.xlsx")
licence1 = pd.read_excel("Data/licence1.xls")
licence2 = pd.read_excel("Data/licence2.xls")
NVOS = pd.read_excel("Data/NVOS.xlsx")
register_of_licenses = pd.read_excel("Data/register_of_licenses.xlsx")
reg_pollutant = pd.read_excel("Data/reg_pollutant.xlsx")

In [6]:
egrul.head()

,№ п/п (произв. площадки),"Наименование юридического лица (филиала по субъекту Российской Федерации), фамилия, имя, отчество индивидуального предпринимателя",Юридический адрес / фактический адрес месторасположения,Идентификационный номер налогоплательщика,Номер и дата выдачи разрешения на выброс вредных (загрязняющих) веществ в атмосферный воздух,Разрешенный выброс вредных (загрязняющих) веществ в пределах утвержденных нормативов ПДВ,(разбивка по веществам в тоннах),Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,(район города Челябинска определен из располож...,NaN,NaN,NaN,аммиак,сероводород,формальдегид,фенол,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,3,NaN,NaN,5,6,7,8,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Челябинский городской округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Калининский район,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,ОАО «Инженерный центр энергетики Урала»,"620075, г. Екатеринбург, ул. Первомайская, д. ...",6.660002e+09,№59 от 22.04.2013,0.021391,Х,Х,Х,Х,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
licence1.head()

,№ п/п,Наименование лицензирующего органа,"Полное и (в случае, если имеется) сокращенное наименование, в том числе фирменное наименование, и организационно-правовая форма юридического лица, адрес его места нахождения, адреса мест осуществления лицензируемого вида деятельности, государственный регистрационный номер записи о создании юридического лица",Unnamed: 3,Unnamed: 4,Unnamed: 5,ИНН,Даты внесения в реестр лицензий сведений о лицензиате\n,Срок действия лицензии,Номер и дата регистрации лицензии,...,Unnamed: 15,Unnamed: 16,"Лицензируемый вид деятельности с указанием выполняемых работ, оказываемых услуг, составляющих лицензируемый вид деятельности",Сведения об адресах мест оосуществления деятельности,"Основания и даты проведения проверок лицензиатов и реквизиты актов, составленных по результатам проведенных проверок",Unnamed: 20,Даты и реквизиты выданных постановлений о назначении административных наказаний в виде административного приостановления деятельности лицензиатов,"Основания, даты вынесения решений лицензирующего органа о приостановлении, о возобновлении действия лицензий и реквизиты таких решений","Основания, даты вынесения решений суда об аннулировании лицензий и реквизиты таких решений",Примечание
0,NaN,NaN,"Наименование предприятия (полное, сокращенное,...",Адрес,Телефон,ОГРН,NaN,NaT,NaN,NaN,...,Код отхода по ФККО,Класс опасности отхода,NaN,NaN,номер и дата проверки,номер и дата выездной проверки,NaN,NaN,NaN,NaN
1,1.0,Управление Росприроднадзора по Челябинской обл...,"ОАО ""Челябинский завод прифилированного стальн...","г.Челябинск, ул.Валдайская, 7",(351)771-09-09,102740232049,7.447015e+09,NaT,13.11.2010-13.11.2015,серия 74 № 00001,...,3533010013011,1,размещение (хранение),"454081,Челябинск, ул. Валдайская, 7",NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Управление Росприроднадзора по Челябинской обл...,"ОАО ""Челябинский завод прифилированного стальн...","г.Челябинск, ул.Валдайская, 7",(351)771-09-09,102740232049,7.447015e+09,NaT,13.11.2010-13.11.2015,серия 74 № 00001,...,9211010113012,2,"транспортировка, размещение (хранение)","454081,Челябинск, ул. Валдайская, 7",NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Управление Росприроднадзора по Челябинской обл...,"ОАО ""Челябинский завод прифилированного стальн...","г.Челябинск, ул.Валдайская, 7",(351)771-09-09,102740232049,7.447015e+09,NaT,13.11.2010-13.11.2015,серия 74 № 00001,...,5410020202033,3,"транспортировка, размещение (хранение)","454081,Челябинск, ул. Валдайская, 7",NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,Управление Росприроднадзора по Челябинской обл...,"ОАО ""Челябинский завод прифилированного стальн...","г.Челябинск, ул.Валдайская, 7",(351)771-09-09,102740232049,7.447015e+09,NaT,13.11.2010-13.11.2015,серия 74 № 00001,...,5410020702033,3,"транспортировка, размещение (хранение)","454081,Челябинск, ул. Валдайская, 7",NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
licence2.head()

,Unnamed: 0,Наименование лицензирующего органа,"Полное и (в случае, если имеется) сокращенное наименование, в том числе фирменное наименование, и организационно-правовая форма юридического лица, адрес его места нахождения, адреса мест осуществления лицензируемого вида деятельности, государственный регистрационный номер записи о создании юридического лица",Unnamed: 3,Unnamed: 4,Unnamed: 5,ИНН,Даты внесения в реестр лицензий сведений о лицензиате\n,Срок действия лицензии,Номер и дата регистрации лицензии,...,Unnamed: 15,Unnamed: 16,"Лицензируемый вид деятельности с указанием выполняемых работ, оказываемых услуг, составляющих лицензируемый вид деятельности",Сведения об адресах мест оосуществления деятельности,"Основания и даты проведения проверок лицензиатов и реквизиты актов, составленных по результатам проведенных проверок",Unnamed: 20,Даты и реквизиты выданных постановлений о назначении административных наказаний в виде административного приостановления деятельности лицензиатов,"Основания, даты вынесения решений лицензирующего органа о приостановлении, о возобновлении действия лицензий и реквизиты таких решений","Основания, даты вынесения решений суда об аннулировании лицензий и реквизиты таких решений",Примечание
0,NaN,NaN,"Наименование предприятия (полное, сокращенное,...",Адрес,Телефон,ОГРН,NaN,NaN,NaN,NaN,...,Код отхода по ФККО,Класс опасности отхода,NaN,NaN,номер и дата проверки,номер и дата выездной проверки,NaN,NaN,NaN,NaN
1,612.0,Уральское межрегиональное управление Росприрод...,ИП Киселюк Владимир Михайлович,"Златоуст, ул.просвещения, 18",(3513) 66-54-40,304740410500024,7.404004e+11,2019-11-14 00:00:00,бессрочно,(74)-8573-Т,...,9 20 110 01 53 2,II,транспортирование отходов II класса опасности,"456208, Челябинская область, г. Златоуст, ул. ...",32-Ч от 01.11.2019,44-Ч от 13.11.2019,NaN,NaN,NaN,NaN
2,612.0,Уральское межрегиональное управление Росприрод...,ИП Киселюк Владимир Михайлович,"Златоуст, ул.просвещения, 18",(3513) 66-54-40,304740410500024,7.404004e+11,2019-11-14 00:00:00,бессрочно,(74)-8573-Т,...,9 20 110 02 52 3,III,транспортирование отходов III класса опасности,"456208, Челябинская область, г. Златоуст, ул. ...",32-Ч от 01.11.2019,44-Ч от 13.11.2019,NaN,NaN,NaN,NaN
3,612.0,Уральское межрегиональное управление Росприрод...,ИП Киселюк Владимир Михайлович,"Златоуст, ул.просвещения, 18",(3513) 66-54-40,304740410500024,7.404004e+11,2019-11-14 00:00:00,бессрочно,(74)-8573-Т,...,9 20 110 03 51 3,III,транспортирование отходов III класса ...,"456208, Челябинская область, г. Златоуст, ул. ...",32-Ч от 01.11.2019,44-Ч от 13.11.2019,NaN,NaN,NaN,NaN
4,612.0,Уральское межрегиональное управление Росприрод...,ИП Киселюк Владимир Михайлович,"Златоуст, ул.просвещения, 18",(3513) 66-54-40,304740410500024,7.404004e+11,2019-11-14 00:00:00,бессрочно,(74)-8573-Т,...,9 21 110 01 50 4,IV,Транспортирование отходов IV класса опасности,"456208, Челябинская область, г. Златоуст, ул. ...",32-Ч от 01.11.2019,44-Ч от 13.11.2019,NaN,NaN,NaN,NaN


In [24]:
NVOS.head()

,№ п/п,Код объекта НВОС,Наименование объекта,ОКТМО,Местонахождение объекта,Наименование\nэксплуатирующей организации,Юридический адрес,ИНН,ОГРН,"Орган, выдавший свидетельство",...,Дата окончания разрешения на размещение отходов,Кол-во объектов размещения отходов,"Суммарная масса отходов, т/год","Выбросы парниковых газов (СО2), т/год",Дата постановки на учет,Дата актуализации сведений,Дата исключения из реестра,Дата ввода в эксплуатацию,Водоохранная зона,Глубоководные выпуски
0,1,75-0274-001025-П,ООО Полипласт,75701315.0,"г Челябинск, ул Молодогвардейцев, д 2","Общество с ограниченной ответственностью ""Поли...","454014, Челябинск, г Челябинск, ул Молодогвард...",7448174764,1157448000379,Министерство экологии Челябинской области,...,NaN,0,NaN,0.000,2016-12-19,2021-04-29,NaN,1978-01-01,Нет,NaN
1,2,75-0274-001032-П,"ОАО ""Кыштымский абразивный завод""",75734000.0,"Челябинская область, г. Кыштым, ул. Станционна...","Открытое акционерное общество ""Кыштымский абра...","456873, г. Кыштым, ул. Станционная, 1 А",7413007498,1027400827904,Министерство экологии Челябинской области,...,NaN,0,NaN,0.000,2016-12-22,NaN,NaN,1988-01-01,Нет,NaN
2,3,75-0274-001326-П,Производственная территория,75626425.0,"456844, Челябинская область, Каслинский район,...","Общество с ограниченной ответсвенностью ""Щерба...","456844, Челябинская область, Каслинский р-н, с...",7459001259,1127459001394,Министерство экологии Челябинской области,...,NaN,0,NaN,0.000,2017-02-13,NaN,NaN,1916-07-01,Нет,NaN
3,4,75-0174-001001-П,Промплощадка Коркинский экскаваторо-вагонорем...,75633101.0,"456550, РФ, Челябинская область, Коркинский ра...","Общество с ограниченной ответственностью ""Корк...","456550, РФ, Челябинская область, Коркинский ра...",7412012791,1087412001050,Министерство экологии Челябинской области,...,NaN,0,NaN,5321.094,2016-12-08,2020-12-04,NaN,1944-07-01,Нет,NaN
4,5,75-0274-001036-П,"ЧОБУ ""Каслинское лесничество""",75626101.0,"Челябинская область, г. Касли, ул. Ленина, 15","Челябинское областное бюджетное учреждение ""Ка...","456835, г. Касли, ул. Ленина, 15",7402009920,1077402001060,Министерство экологии Челябинской области,...,NaN,0,NaN,0.000,2016-12-23,NaN,NaN,2008-01-01,Нет,NaN


In [25]:
register_of_licenses.head()

,"Реестр лицензий, выданных на заготовку, хранение, переработку и реализацию лома черных металлов, цветных металлов на территории Челябинской области",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,№ п/п,Регистрационный номер,"Серия лицензии, дата выдачи",Наименование лицензирующего органа,Основания решения\n о\nвыдаче лицензии,Основание и дата прекращения\nдействия лицензии\n,"Наименование предприятия (полное и (в случае, ...",ИНН ОГРН,Вид работ,Основания и даты проведения проверок лицензиат...,"Основания, даты вынесения решений лицензирующе...",Юридический адрес,Телефон,Место осуществления деятельности
2,1,001,Ч 740002 23.12.2011,NaN,Приказ № 140 от 23.12.2011,NaN,ООО «УралВторМет»,7415073915 1117415004145,черный лом,"Приказ № 210-ПЛ от 13.10.2014, Акт проверки № ...",NaN,"г. Миасс, ул. МЖК, д. 1",8(3513)56-62-82,"г. Миасс, ул. Силовая, 1"
3,2,002,Ч 740007\n23.12.2011\n,NaN,NaN,Лицензия аннулирована по заявлению лицензиата ...,"ООО ""ЧелМет""\n\n",7451328319\n1117451015120\n,черный лом\nцветной лом\n,NaN,NaN,"г. Челябинск,\nТроицкий тракт, д. 3а\n",218-41-59,"г. Челябинск,\nТроицкий тракт, 3а\n"
4,3,003,Ч 740005\n23.12.2011\n,NaN,NaN,Лицензия аннулирована по заявлению лицензиата ...,"ООО ""Златрем""\n\n\n",7404029897\n1027400576939\n,черный лом\nцветной лом\n,NaN,NaN,"г. Златоуст,\nул. Карла Маркса, д. 49, кв.33\n",(3513)\n67-41-99\n,"1. г. Златоуст,\nул. Лесная, 2\n2. г. Сатка, О..."


In [26]:
reg_pollutant.head()

,№ п/п (произв. площадки),"Наименование юридического лица (филиала по субъекту Российской Федерации), фамилия, имя, отчество индивидуального предпринимателя",Юридический адрес / фактический адрес месторасположения,Идентификационный номер налогоплательщика,Номер и дата выдачи разрешения на выброс вредных (загрязняющих) веществ в атмосферный воздух,Разрешенный выброс вредных (загрязняющих) веществ в пределах утвержденных нормативов ПДВ,(разбивка по веществам в тоннах),Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,(район города Челябинска определен из располож...,NaN,NaN,NaN,аммиак,сероводород,формальдегид,фенол,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,3,NaN,NaN,5,6,7,8,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Челябинский городской округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Калининский район,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,ОАО «Инженерный центр энергетики Урала»,"620075, г. Екатеринбург, ул. Первомайская, д. ...",6.660002e+09,№59 от 22.04.2013,0.021391,Х,Х,Х,Х,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data = 

In [ ]:
X = data
y = data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75)

In [18]:
parameters = {'max_depth':range(3,100)}
clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4)
clf.fit(X=x, y=y)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
precision = precision_score(y_test, pred, average='micro')

In [ ]:
clf.score(X_test, y_test)